In [31]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import random


In [32]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])
a = open('processed.cleveland.data', 'r')
c = a.readlines()
random.shuffle(c) #shuffle data before using
n_val = []
n_keys = []
d = []
for elem in c:
    elem = elem[:-1]
    elem = list(map(float,elem.split(',')))
    n_val.append(elem[:-1])
    n_keys.append(elem[-1]) #mb ad
rot = [[] for i in range(len(n_val[0]))]
mean = []
std = []
for i in range(len(n_val)):
    for j in range(len(n_val[i])):
        rot[j].append(n_val[i][j])
#print(len(rot[0]))
for elem in rot:
    #print(elem)
    new_elem = np.array(elem)
    #print(np.std(new_elem))
    #print(new_elem.shape)
    std.append(np.std(new_elem))
    mean.append(np.mean(new_elem))
    #elem = [int(elem[i]*10) for i in range(len(elem))] 
for i in range(len(rot)):
    rot[i] = [(rot[i][j] - mean[i])/std[i] for j in range(len(elem))]
#print(rot[0])
rot = np.array(rot)
n_val = rot.T     #all of the previous 20 lines normalises our array


#true-false
for i in range(len(n_keys)):
    if n_keys[i] > 0:
        n_keys[i] = 1
print(n_keys)

[1, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 1, 1, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1, 0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 1, 0.0, 0.0, 1, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 1, 0.0, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 1, 0.0, 1, 1, 1, 1, 0.0, 0.0, 0.0, 1, 1, 1, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 0.0, 0.0, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 1, 1, 0.0, 1, 0.0, 0.0, 1, 1, 0.0, 1, 1, 1, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 1, 1, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 1, 1, 1, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 1, 0.0, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 1, 0.0, 1, 0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1, 0.0, 1, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0, 1, 1, 1, 0.0, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 1, 1, 1, 0.0, 1, 1, 1, 1, 1, 0.0, 0.0, 1, 0.0, 1, 1, 1, 0.0, 0.0, 1, 0.0, 1, 0.0, 0.0, 1, 1, 

In [33]:
#73 - test size
#230 - train size
t = 230
te = 73
a = random.sample(range(303), t)
val, keys, te_val, te_keys = [],[],[],[]
for i in range(len(n_val)):
    if i in a:
        val.append(n_val[i])
        keys.append(n_keys[i])
    else:
        te_val.append(n_val[i])                       #train-test 
        te_keys.append(n_keys[i])

val = np.array(val)
keys = np.array(keys)
#val = val.astype(float)
#keys = keys.astype(float)

te_val = np.array(te_val)
te_keys = np.array(te_keys)
#keys.shape
val.shape



#now we need to create a cycle for bootstrap


(226, 13)

In [36]:
number_of_models = 300
voting_list = []
for trap in range(number_of_models):
    trap_val, trap_keys = [], []
    for i in range(len(val)//3 * 2):
        ind = random.randrange(len(val))
        trap_val.append(val[ind])
        trap_keys.append(keys[ind])
    #print(len(trap_keys))
    input_size = 13
    hidden_sizes = [128, 128, 64]
    output_size = 2

    model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[1], output_size),
                          nn.LogSoftmax(dim=1))
    #print(model) #model with 3 layers, first 2 with relu, last with logistic curve
    criterion = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.7)
    time0 = time()
    epochs = 30
    for e in range(epochs):
        running_loss = 0
        for i in range(len(trap_val)):
            # Training pass
            optimizer.zero_grad()
            temp_1 = [trap_val[i]]
            temp0 = np.array(temp_1)
            temp = torch.from_numpy(temp0)
            temp1 = temp.float()
            #print(temp1)
            output = model(temp1)    #DOES NOT WORK. MODEL RETURNS NAN TENSOR
            #print(output)
            k = [trap_keys[i]]
            k = np.array(k)
            k = torch.from_numpy(k)
            k = k.long()
            loss = criterion(output, k)

            #This is where the model learns by backpropagating
            loss.backward()

            #And optimizes its weights here
            optimizer.step()

            #print(loss.item())
            running_loss += loss.item()
        else:
            print("Epoch {} - Training loss: {}".format(e, running_loss/len(val)))

    print("\nTraining Time (in minutes) =",(time()-time0)/60)
    
    model_answers = []
    for i in range(len(te_val)):
        temp_1 = [te_val[i]]
        temp0 = np.array(temp_1)
        temp = torch.from_numpy(temp0)
        temp1 = temp.float()
        with torch.no_grad():
            logps = model(temp1)
        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        model_answers.append(pred_label)
    voting_list.append(model_answers)


Epoch 0 - Training loss: 0.3943315398904075
Epoch 1 - Training loss: 0.29812574574509554
Epoch 2 - Training loss: 0.24966983277499016
Epoch 3 - Training loss: 0.2243026958292235
Epoch 4 - Training loss: 0.2049366956209945
Epoch 5 - Training loss: 0.18739369060160113
Epoch 6 - Training loss: 0.17116731577923736
Epoch 7 - Training loss: 0.15576436211338546
Epoch 8 - Training loss: 0.14141071316079376
Epoch 9 - Training loss: 0.12859133481402443
Epoch 10 - Training loss: 0.11622961196199522
Epoch 11 - Training loss: 0.10509543147286302
Epoch 12 - Training loss: 0.09490007268387486
Epoch 13 - Training loss: 0.08590541904182039
Epoch 14 - Training loss: 0.07772058408304434
Epoch 15 - Training loss: 0.07082898997795326
Epoch 16 - Training loss: 0.064418136470546
Epoch 17 - Training loss: 0.05906984383994241
Epoch 18 - Training loss: 0.053709124284020526
Epoch 19 - Training loss: 0.04913666243657596
Epoch 20 - Training loss: 0.04469340050060151
Epoch 21 - Training loss: 0.04082298978853708
Ep

Epoch 23 - Training loss: 0.02967420083353346
Epoch 24 - Training loss: 0.026194874999871203
Epoch 25 - Training loss: 0.0234032922763987
Epoch 26 - Training loss: 0.020995479559395878
Epoch 27 - Training loss: 0.018971552043232223
Epoch 28 - Training loss: 0.01711361193311784
Epoch 29 - Training loss: 0.015697593705234533

Training Time (in minutes) = 0.12442739009857177
Epoch 0 - Training loss: 0.4136770801470343
Epoch 1 - Training loss: 0.32019664391677466
Epoch 2 - Training loss: 0.2683314835215894
Epoch 3 - Training loss: 0.2343511842871167
Epoch 4 - Training loss: 0.20787205644553544
Epoch 5 - Training loss: 0.18610062667908378
Epoch 6 - Training loss: 0.16741463444705648
Epoch 7 - Training loss: 0.15073090656163415
Epoch 8 - Training loss: 0.13584673005282163
Epoch 9 - Training loss: 0.12178916678491818
Epoch 10 - Training loss: 0.1093559147983499
Epoch 11 - Training loss: 0.09763058035503941
Epoch 12 - Training loss: 0.08690947721973807
Epoch 13 - Training loss: 0.0771649163955

Epoch 16 - Training loss: 0.042821880839390446
Epoch 17 - Training loss: 0.0390837019816521
Epoch 18 - Training loss: 0.03573455374329862
Epoch 19 - Training loss: 0.03269559649678221
Epoch 20 - Training loss: 0.0301930554765115
Epoch 21 - Training loss: 0.02757306800340486
Epoch 22 - Training loss: 0.02558442559302405
Epoch 23 - Training loss: 0.02394679826086572
Epoch 24 - Training loss: 0.021998572033781125
Epoch 25 - Training loss: 0.020527024022497082
Epoch 26 - Training loss: 0.01911990850643534
Epoch 27 - Training loss: 0.01782367857132125
Epoch 28 - Training loss: 0.016506793346897496
Epoch 29 - Training loss: 0.015592255098783774

Training Time (in minutes) = 0.10227546294530233
Epoch 0 - Training loss: 0.38584810600871533
Epoch 1 - Training loss: 0.24952706844603592
Epoch 2 - Training loss: 0.1915469967587068
Epoch 3 - Training loss: 0.1676573466635268
Epoch 4 - Training loss: 0.15108230664854275
Epoch 5 - Training loss: 0.13696915082493386
Epoch 6 - Training loss: 0.12388535

Epoch 8 - Training loss: 0.13634772968123846
Epoch 9 - Training loss: 0.12420314650267128
Epoch 10 - Training loss: 0.11304928205901035
Epoch 11 - Training loss: 0.10162153599795819
Epoch 12 - Training loss: 0.09189479174129758
Epoch 13 - Training loss: 0.08296772075422736
Epoch 14 - Training loss: 0.07436829524440891
Epoch 15 - Training loss: 0.06696745423789607
Epoch 16 - Training loss: 0.05966097613064455
Epoch 17 - Training loss: 0.05304234750317734
Epoch 18 - Training loss: 0.0480503788637712
Epoch 19 - Training loss: 0.04262502565547704
Epoch 20 - Training loss: 0.038660599151205406
Epoch 21 - Training loss: 0.034804446838729984
Epoch 22 - Training loss: 0.031182464350429547
Epoch 23 - Training loss: 0.028274712316876116
Epoch 24 - Training loss: 0.025462783448716254
Epoch 25 - Training loss: 0.022835760003515813
Epoch 26 - Training loss: 0.02110548955243224
Epoch 27 - Training loss: 0.018907537692804666
Epoch 28 - Training loss: 0.01759130155946482
Epoch 29 - Training loss: 0.01

Epoch 0 - Training loss: 0.4013260600313676
Epoch 1 - Training loss: 0.2770359836584699
Epoch 2 - Training loss: 0.20519243848955143
Epoch 3 - Training loss: 0.17054546954299826
Epoch 4 - Training loss: 0.14676743406010173
Epoch 5 - Training loss: 0.12747396726304958
Epoch 6 - Training loss: 0.11137234266615315
Epoch 7 - Training loss: 0.09767877747277952
Epoch 8 - Training loss: 0.0859575314184319
Epoch 9 - Training loss: 0.07549685374839761
Epoch 10 - Training loss: 0.06668315458625483
Epoch 11 - Training loss: 0.05841776235318932
Epoch 12 - Training loss: 0.05120033419472498
Epoch 13 - Training loss: 0.044840326871907706
Epoch 14 - Training loss: 0.038720058456909755
Epoch 15 - Training loss: 0.03366996810466042
Epoch 16 - Training loss: 0.029298853285670105
Epoch 17 - Training loss: 0.025378036400659443
Epoch 18 - Training loss: 0.022343985688663682
Epoch 19 - Training loss: 0.019467982776911825
Epoch 20 - Training loss: 0.017411287711674734
Epoch 21 - Training loss: 0.015372916665

...

Epoch 19 - Training loss: 0.030747172802563855
Epoch 20 - Training loss: 0.027179421779730862
Epoch 21 - Training loss: 0.024652401486154312
Epoch 22 - Training loss: 0.0221280971739654
Epoch 23 - Training loss: 0.020353702353702343
Epoch 24 - Training loss: 0.018207963543845105
Epoch 25 - Training loss: 0.01684352965497266
Epoch 26 - Training loss: 0.015379675172687703
Epoch 27 - Training loss: 0.014054469479339143
Epoch 28 - Training loss: 0.013021339026046613
Epoch 29 - Training loss: 0.011982154606022252

Training Time (in minutes) = 0.09104117552439371
Epoch 0 - Training loss: 0.4264921938951037
Epoch 1 - Training loss: 0.3297844466843964
Epoch 2 - Training loss: 0.26636459816873603
Epoch 3 - Training loss: 0.23158795044106323
Epoch 4 - Training loss: 0.20670735756083664
Epoch 5 - Training loss: 0.18572950628956994
Epoch 6 - Training loss: 0.1663175959488278
Epoch 7 - Training loss: 0.14942894706582915
Epoch 8 - Training loss: 0.13349421481669888
Epoch 9 - Training loss: 0.1187724

Epoch 11 - Training loss: 0.10636959865062644
Epoch 12 - Training loss: 0.09795464344470584
Epoch 13 - Training loss: 0.09102276812421956
Epoch 14 - Training loss: 0.08420659199356904
Epoch 15 - Training loss: 0.0781417472654233
Epoch 16 - Training loss: 0.07239429763027887
Epoch 17 - Training loss: 0.06739313572210352
Epoch 18 - Training loss: 0.062388306294300554
Epoch 19 - Training loss: 0.05774825935735011
Epoch 20 - Training loss: 0.05342010739965256
Epoch 21 - Training loss: 0.04933162951196621
Epoch 22 - Training loss: 0.04525875743415875
Epoch 23 - Training loss: 0.04145900357750845
Epoch 24 - Training loss: 0.038071868694631104
Epoch 25 - Training loss: 0.0348796559029928
Epoch 26 - Training loss: 0.03180260417636625
Epoch 27 - Training loss: 0.028704913812072692
Epoch 28 - Training loss: 0.026229568151941007
Epoch 29 - Training loss: 0.023852376324873924

Training Time (in minutes) = 0.09180790980656942
Epoch 0 - Training loss: 0.3931075424219655
Epoch 1 - Training loss: 0.22

Epoch 3 - Training loss: 0.21350496189578114
Epoch 4 - Training loss: 0.19616314287161438
Epoch 5 - Training loss: 0.18095543724381719
Epoch 6 - Training loss: 0.16715487775359864
Epoch 7 - Training loss: 0.1539377495749499
Epoch 8 - Training loss: 0.1415120231826528
Epoch 9 - Training loss: 0.1295295472669444
Epoch 10 - Training loss: 0.11754662514003215
Epoch 11 - Training loss: 0.1062979010783447
Epoch 12 - Training loss: 0.09570718747546275
Epoch 13 - Training loss: 0.08573099608442279
Epoch 14 - Training loss: 0.076783803963337
Epoch 15 - Training loss: 0.06861681382934248
Epoch 16 - Training loss: 0.061342737000678625
Epoch 17 - Training loss: 0.05492873393871426
Epoch 18 - Training loss: 0.049190444120413616
Epoch 19 - Training loss: 0.044217693051362465
Epoch 20 - Training loss: 0.039624490127912276
Epoch 21 - Training loss: 0.03572160312577214
Epoch 22 - Training loss: 0.03213785632882984
Epoch 23 - Training loss: 0.028939509109626664
Epoch 24 - Training loss: 0.02623767395385

Epoch 26 - Training loss: 0.022239359445530394
Epoch 27 - Training loss: 0.01963403474478672
Epoch 28 - Training loss: 0.01762586937222299
Epoch 29 - Training loss: 0.015761209337931504

Training Time (in minutes) = 0.09144120216369629
Epoch 0 - Training loss: 0.39404949578827464
Epoch 1 - Training loss: 0.26721692986918233
Epoch 2 - Training loss: 0.21306346061812564
Epoch 3 - Training loss: 0.1865765923300379
Epoch 4 - Training loss: 0.1677258215407816
Epoch 5 - Training loss: 0.15196911953966571
Epoch 6 - Training loss: 0.1376077960197921
Epoch 7 - Training loss: 0.12458942936109049
Epoch 8 - Training loss: 0.11253494794652551
Epoch 9 - Training loss: 0.10132106175374207
Epoch 10 - Training loss: 0.09020903642013497
Epoch 11 - Training loss: 0.08033323668815176
Epoch 12 - Training loss: 0.07127735375190918
Epoch 13 - Training loss: 0.0628903425491782
Epoch 14 - Training loss: 0.05577850761119005
Epoch 15 - Training loss: 0.04932053948276216
Epoch 16 - Training loss: 0.04358914481933

Epoch 18 - Training loss: 0.04112363726678936
Epoch 19 - Training loss: 0.03697455115927992
Epoch 20 - Training loss: 0.03417095272937879
Epoch 21 - Training loss: 0.030939884571085458
Epoch 22 - Training loss: 0.027832291134364143
Epoch 23 - Training loss: 0.025712938477940057
Epoch 24 - Training loss: 0.023307851473635535
Epoch 25 - Training loss: 0.021350164364309393
Epoch 26 - Training loss: 0.01968818848649824
Epoch 27 - Training loss: 0.017869947804122967
Epoch 28 - Training loss: 0.01643510453726838
Epoch 29 - Training loss: 0.015096998351041951

Training Time (in minutes) = 0.09075782696406047
Epoch 0 - Training loss: 0.38298206598357815
Epoch 1 - Training loss: 0.26105299109049074
Epoch 2 - Training loss: 0.20895305902499342
Epoch 3 - Training loss: 0.18283651197464124
Epoch 4 - Training loss: 0.16443822658167476
Epoch 5 - Training loss: 0.14962763087880648
Epoch 6 - Training loss: 0.13689870741305576
Epoch 7 - Training loss: 0.125511259933351
Epoch 8 - Training loss: 0.115075

In [57]:
#here we are going out of the cycle and compare results via voting




correct_count, all_count = 0, 0    
norm_c, norm_true = 0,0
dis_c, dis_true = 0,0

for i in range(len(te_val)):
    sum_prediction = 0
    for j in range(len(voting_list)):
        sum_prediction += voting_list[j][i]
    if sum_prediction >= number_of_models//1.5:  #2 by default
        prediction = 1
    else:
        prediction = 0
    true_label = te_keys[i]
    if(true_label == prediction):
        correct_count += 1
        if true_label == 0:
            norm_true += 1
        else:
            dis_true += 1
    if true_label == 0:
        norm_c += 1
    else:
        dis_c += 1        
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

#0.9014084507042254

Number Of Images Tested = 71

Model Accuracy = 0.9014084507042254


In [58]:
#true-false 80 percent need 90
print("Number Of norm =", norm_c)
print("\nModel Accuracy norm=", (norm_true/norm_c))
print("Number Of dis =", dis_c)
print("\nModel Accuracy dis=", (dis_true/dis_c))

Number Of norm = 40

Model Accuracy norm= 0.925
Number Of dis = 31

Model Accuracy dis= 0.8709677419354839


In [15]:














torch.save(model, './my_mnist_model.pt') 